In [ ]:
%pyspark
from pyspark import SparkFiles
# Load a shoes data from S3 into a DataFrame

shoes_df = spark.read.option('header', 'true').csv("s3a://amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz", inferSchema=True, sep='\t')
shoes_df.show(10)

In [ ]:
%pyspark
from pyspark.sql.functions import to_date

date_df = shoes_df.withColumn("date", to_date("review_date", "yyyy-mm-dd"))

In [ ]:
%pyspark
from pyspark.sql.functions import col

review_df = date_df.select(["review_id", "customer_id", "product_id", "product_parent", col("date").alias("review_date")])
review_df.show(10)

In [ ]:
%pyspark

review_df.count()

In [ ]:
%pyspark

products_df = date_df.select(["product_id", "product_title"]).distinct()

In [ ]:
%pyspark

products_df.count()

In [ ]:
%pyspark

customer_df = date_df.groupBy("customer_id").count().orderBy("customer_id")

In [ ]:
%pyspark
customer_df = customer_df.withColumnRenamed("count", "customer_count")
customer_df.show(10)

In [ ]:
%pyspark
customer_df.count()

In [ ]:
%pyspark

vine_df = date_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show(10)

In [ ]:
%pyspark

vine_df.count()

In [ ]:
%pyspark

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:<port>/<db_name>"
config = {"user":"<usrname>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
%pyspark
# Append DataFrame to review_id_table in RDS
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
%pyspark

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
%pyspark
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
%pyspark
# Write dataframe to vine_table table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
%pyspark
# Write dataframe to vine_table table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table_shoes', mode=mode, properties=config)